In [ ]:
import pandas as pd

df = pd.read_csv("/Users/jeroen.vander.putten/Development/Learning/Python/github_portfolio_jeroen/project_7k_books/books.csv")
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.axes()
sns.heatmap(df.isna().transpose(), cbar=False, ax=ax)

plt.xlabel("Rows")
plt.ylabel("Missing values")

plt.show()

In [ ]:
import numpy as np

current_year = pd.Timestamp.now().year
df["age_of_books"] = current_year - pd.to_numeric(df["published_year"], errors="coerce")
df["missing_description"] = np.where(df["description"].isna(), 1, 0)

In [ ]:
columns_of_interest = ["num_pages", "age_of_books", "missing_description", "average_rating"]
correlation_matrix = df[columns_of_interest].corr(method="spearman")

sns.set_theme(style="white")
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm",
                      cbar_kws={"label": "Correlation coefficient"})
heatmap.set_title("Correlation heatmap")
plt.show()

In [ ]:
cols = ["description", "num_pages", "average_rating", "published_year"]
missing_mask = df[cols].isna().any(axis=1)  # true where any of these are NaN
print(df.shape[0])
print(missing_mask.shape[0])
round(df[missing_mask].shape[0]/ df.shape[0], 2)  # percentage 

In [ ]:
# remove missings
df = df[~missing_mask].copy()
df.shape[0]

In [ ]:
df["categories"].value_counts().reset_index().sort_values("count", ascending=False)

,categories,count
0,Fiction,2523
1,Juvenile Fiction,534
2,Biography & Autobiography,391
3,History,258
4,Literary Criticism,164
...,...,...
526,Courage,1
527,Otherland (Imaginary place),1
528,Animals,1
529,Children of divorced parents,1


In [25]:
df["words_in_description"] = df["description"].str.split().str.len()
df["words_in_description"].describe()

count    6507.000000
mean       66.325803
std        66.120291
min         1.000000
25%        27.000000
50%        40.000000
75%        82.000000
max       920.000000
Name: words_in_description, dtype: float64

In [27]:
df = df[df["words_in_description"] >= 25].copy()
df.shape[0]

5197

In [28]:
df["title_and_subtitle"] = (
    np.where(df["subtitle"].isna(), df["title"],
             df[["title", "subtitle"]].astype(str).agg(": ".join, axis=1))
).copy()
df["title_and_subtitle"]

0                                                  Gilead
1                                   Spider's Web: A Novel
3                                          Rage of angels
4                                          The Four Loves
5                                     The Problem of Pain
                              ...                        
6802                                    Mistaken Identity
6803                                  Journey to the East
6804    The Monk Who Sold His Ferrari: A Fable About F...
6805       I Am that: Talks with Sri Nisargadatta Maharaj
6808                             The Berlin Phenomenology
Name: title_and_subtitle, Length: 5197, dtype: object

In [29]:
df["tagged_description"] = df[["isbn13", "description"]].astype(str).agg(" ".join, axis=1).copy()
df["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
3       9780006178736 A memorable, mesmerizing heroine...
4       9780006280897 Lewis' work on the nature of lov...
5       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
6802    9788172235222 On A Train Journey Home To North...
6803    9788173031014 This book tells the tale of a ma...
6804    9788179921623 Wisdom to Create a Life of Passi...
6805    9788185300535 This collection of the timeless ...
6808    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [31]:
df_clean = df.drop(["subtitle", "missing_description", "age_of_books", "words_in_description"], axis=1)
df_clean.to_csv("books_cleaned.csv", index = False)